In [15]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', 1000)
from IPython.display import display, HTML
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from IPython.display import HTML

In [16]:
def splice_list(row):
        if row['num_responses_outputted'] > row['num_responses']:
            return row['cleaned_responses'][:row['num_responses']]
        else:
            return row['cleaned_responses']

def get_num_words(lst):
    if lst:  # Checking if the list is not empty
        return sum(len([i for i, x in enumerate(s) if x == " "]) + 1 for s in lst) / len(lst)
    return 0  # Return 0 for an empty list to avoid division by zero

def process_LLM_outputs(df):
    df = df[df["temp"] <= 1].reset_index(drop=True)        # high temp responses are bad

    num_reps = 5
    df["rep"] = [i+1 for i in range(num_reps)] * (len(df)//num_reps)

    cleaned_responses = []
    for index, row in df.iterrows():
        response = row["response"]
        response = response.replace(", ", ",")
        response = response.replace(",,", ",")
        response = response.replace("_", " ")
        for torem in ["[", ".", "]"]:
            response = response.replace(torem, "")
        response = re.sub(r'\s*\([^)]*\)|\s*\([^)]*$', '', response)
        cleaned_responses.append(response.lower())

    df["cleaned_responses"] = cleaned_responses
    df['cleaned_responses'] = df['cleaned_responses'].apply(lambda x: x.split(','))

    df["num_responses_outputted"] = df['cleaned_responses'].apply(len)

    # Apply the function to adjust the 'list' column in place
    df['cleaned_responses'] = df.apply(splice_list, axis=1)
    df["num_responses_outputted_aftersplice"] = df['cleaned_responses'].apply(len)

    df["num_responses_correct"] = df["num_responses"] == df["num_responses_outputted_aftersplice"]
    df["num_responses_>=19"] = df["num_responses_outputted"] >= 19

    df["num_words_outputted"] = df['cleaned_responses'].apply(get_num_words)
    df["num_words_correct"] = (df["num_words_outputted"] >= 1) & (df["num_words_outputted"] <= 5)

    generate_HTML(df)
    save_outputs(df)

def generate_HTML(df):
     # Initialize an empty string to accumulate HTML content
    html_output = ""

    for model in df["model"].unique():
        df_toshow = df[df["model"] == model][["temp", "rep", "cleaned_responses", "num_responses_>=19", "num_words_correct"]]

        styled_df = df_toshow.style.format("{:.2f}", subset=["temp"]).hide_index().set_properties(**{'text-align': 'left', 'vertical-align': 'top'})

        # Custom CSS to make the DataFrame scrollable, defined once outside the loop
        scrollable_table_style = """
        <style>
            .scrollable_dataframe {
                display: block;
                overflow-x: auto;
                white-space: nowrap;
            }
        </style>
        """

        # Wrap the styled DataFrame in a div with the 'scrollable_dataframe' class
        scrollable_df_html = (
            '<div class="scrollable_dataframe">' +
            styled_df.render() +
            '</div>'
        )

        # Add the model name, the calculated percentage, and the DataFrame HTML to the overall HTML output
        html_output += f"<h2>{model}</h2>" + scrollable_df_html + "<br>"

    # Prepend the custom CSS to the entire HTML output so it only needs to be defined once
    html_output = scrollable_table_style + html_output

    # Write the accumulated HTML content to an HTML file
    with open(f"../data_LLM/task{task}_final_output.html", "w") as file:
        file.write(html_output)

    print(f"HTML file has been created and saved to../data_LLM/task{task}_final_output.html")
    assert sum(df["num_responses_>=19"])/len(df) * 100 == 100
    assert sum(df["num_words_correct"])/len(df) * 100 == 100

def save_outputs(df):
    df = df.explode('cleaned_responses').reset_index(drop=True)
    df.drop(columns=["timestamp", "response", "num_responses", "num_responses_outputted", "num_responses_outputted_aftersplice", "num_responses_correct", "num_responses_>=19", "num_words_outputted", "num_words_correct"], inplace=True)
    df.to_csv(f"../data_LLM/results_task{task}_final_foranalysis.csv")
    print(f"csv file has been created and saved to ../data_LLM/results_task{task}_final_foranalysis.csv")

In [17]:
task = 2
df = pd.read_csv(f"../data_LLM/results_task{task}_finalmodels.csv")
process_LLM_outputs(df)

HTML file has been created and saved to../data_LLM/task2_final_output.html
csv file has been created and saved to ../data_LLM/results_task2_final_foranalysis.csv
